To run:
1. On the camera start the Smart Remote Control (https://www.playmemoriescameraapps.com/portal/usbdetail.php?eid=is9104-npia09014_00-f00002) application
2. Connect your computer to the WiFi hotspot created by the camera; after this step the application's screen should change to a loading icon with the text 'Connecting' underneath it
3. Run this Notebook

# Import necessary libraries

In [1]:
import requests, xmltodict, json

from PIL import Image
import requests
from io import BytesIO

import time

# Camera Initialisation

## Connect to camera to get model and UUID
**Note: your camera might have a different ip; after you connect your computer to the camera's WiFi network, look what IP address you have (ex: in Ubuntu run the command 'ip a'). The camera should have the same address but with the last group of digits replaced with a 1. Example: after connecting to the camera you were assigned the ip 192.168.132.143, the camera's ip probably is 192.168.132.1; some cameras might use a different location for the api xml, in this case you will have to scan for open ports (ex: nmap -sT -p0-65535 [camera ip]) and look for the XML perform an M-SEARCH request (see MSEARCH.py)

In [2]:
r = requests.get('http://192.168.122.1:61000/scalarwebapi_dd.xml')
data = xmltodict.parse(r.text)
attributes = ['deviceType', 'friendlyName', 'UDN']

for a in attributes:
    print(f'{a}: {data["root"]["device"][a]}')

deviceType: urn:schemas-upnp-org:device:Basic:1
friendlyName: NEX-5T-Dumi
UDN: uuid:000000001000-1010-8000-7A4B8739984F


Get API version

In [3]:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"getVersions","params":[],"id":1,"version":"1.0"}'))
print(json.loads(r.text))

{'id': 1, 'result': [['1.0']]}


Get application version

In [4]:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"getApplicationInfo","params":[],"id":2,"version":"1.0"}'))
print(json.loads(r.text))

{'id': 2, 'result': ['Smart Remote Control', '1.1.0']}


Get supported methods and their signatures

In [5]:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"getMethodTypes","params":["1.0"],"id":3,"version":"1.0"}'))
for m in json.loads(r.text)['results']:
    print(f'{m}')

['getVersions', [], ['string*'], '1.0']
['getMethodTypes', ['string'], ['string', 'string*', 'string*'], '1.0']
['getApplicationInfo', [], ['string', 'string'], '1.0']
['actTakePicture', [], ['string*'], '1.0']
['awaitTakePicture', [], ['string*'], '1.0']
['startRecMode', [], ['int'], '1.0']
['stopRecMode', [], ['int'], '1.0']
['startLiveview', [], ['int'], '1.0']
['stopLiveview', [], ['int'], '1.0']
['getAvailableApiList', [], ['string*'], '1.0']
['setExposureCompensation', ['int'], ['int'], '1.0']
['getExposureCompensation', [], ['int'], '1.0']
['getSupportedExposureCompensation', [], ['int*', 'int*', 'int*'], '1.0']
['getAvailableExposureCompensation', [], ['int', 'int', 'int'], '1.0']
['setSelfTimer', ['int'], ['int'], '1.0']
['getSelfTimer', [], ['int'], '1.0']
['getSupportedSelfTimer', [], ['int*'], '1.0']
['getAvailableSelfTimer', [], ['int*'], '1.0']
['receiveEvent', ['bool'], ['string', 'bool', 'int', 'int', 'int', 'int', 'string*', 'string*', 'bool*', 'string*', 'string*'], '

# Start the recording / remote control mode
Start the recording mode. After you run this cell the camera should display a live view on its backscreen. 

If you want to **stop** the cording mode run:

r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"stopRecMode","params":[],"id":4,"version":"1.0"}'))
print(json.loads(r.text))

In [6]:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"startRecMode","params":[],"id":4,"version":"1.0"}'))
print(json.loads(r.text))

{'id': 4, 'result': [0]}


List all available APIs

In [7]:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"getAvailableApiList","params":[],"id":5,"version":"1.0"}'))
for m in json.loads(r.text)['result'][0]:
    print(m)

getMethodTypes
getVersions
getApplicationInfo
getAvailableApiList
receiveEvent
startRecMode
stopRecMode
startLiveview
stopLiveview


## Start the liveview
This returns an API endpoint which will provide a stream showing a liveview of the camera

In order to stop the stream run:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"stopLiveview","params":[],"id":6,"version":"1.0"}'))

In [8]:
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"startLiveview","params":[],"id":6,"version":"1.0"}'))
print(json.loads(r.text))

{'id': 6, 'result': ['http://192.168.122.1:8080/liveview/liveview.jpg']}


# Take a picture

When called the API endpoint will take a stillshot, and return the image as raw data; we decode and display the data using Pillow

In [9]:
a = time.time()
r = requests.post('http://192.168.122.1:8080/sony/camera', json=json.loads('{"method":"actTakePicture","params":[],"id":7,"version":"1.0"}'))
b = time.time()
print(f'Request time: {b-a}s')

print(json.loads(r.text))
pic_url = json.loads(r.text)['result'][0][0]

response = requests.get(pic_url)
img = Image.open(BytesIO(response.content))
print(f'Image size: {img.size}')
img

Request time: 5.057635545730591s
{'id': 7, 'result': [['http://192.168.122.1:8080/postview/pict20210103_213924_0.JPG']]}


# View the liveview

The liveview is just a sequence of JPEG images; we read 1024 bytes of data and look for the JPEG file start and file end signatures; the images are displayed as soon as they are available

In [1]:
import urllib
from PIL import Image
import io

In [ ]:
stream = urllib.request.urlopen('http://192.168.122.1:8080/liveview/liveview.jpg')

bytes_received = b''
while True:
    bytes_received += stream.read(1024)
    a = bytes_received.find(b'\xff\xd8\xff', 0)
    b = bytes_received.find(b'\xff\xd9', a+3)
    if a != -1 and b != -1:
        jpg = bytes_received[a:b+2]
        bytes_received = bytes_received[b+2:]
        img = Image.open(io.BytesIO(jpg))
        display(img)